# Experiment // Rectangles

### Preparation
At the beginning, let's import the most important modules and classes from the MedTagger internal API.

In [1]:
import collections
import numpy as np
from medtagger.database import models
from medtagger.ground_truth.algorithms.majority_voting import MajorityVotingAlgorithm
from medtagger.ground_truth.algorithms.gaussian_mixture_models import GaussianMixtureModelsAlgorithm
from medtagger.ground_truth.algorithms.k_means import KMeansAlgorithm
from medtagger.ground_truth.algorithms.dbscan import DBSCANAlgorithm
from medtagger.ground_truth.parsers.rectangle import RectangleLabelElementParser
from medtagger.ground_truth.generator import DataSetGenerator
from medtagger.ground_truth.quality import figures
from medtagger.ground_truth.quality.user_specificity_sensitivity import \
    compute_specificity_and_sensitivity_for_users

In [2]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [8.0, 6.0]
plt.rcParams['figure.dpi'] = 120

In [3]:
SCAN_IDS = [
    '03647f36-d874-480b-aaec-c2c3b9d80092',
    '40761156-99db-4647-9b72-8e449e6cb54c',
    'b750320e-5664-4eff-b4c7-49e4f43ce6ca',
    '8b5d264d-bc2f-458f-ba68-2a987d55deae',
    '7985ea16-d5a4-4bda-92db-43a3cc074216'
]

### Create Generator instance

To create Ground Truth data set, all you need is to define your data set generator and an algorithm that should be used during generation.

In [4]:
# algorithm = MajorityVotingAlgorithm()
# algorithm = GaussianMixtureModelsAlgorithm()
# algorithm = KMeansAlgorithm()
algorithm = DBSCANAlgorithm()
generator = DataSetGenerator(algorithm)

In [5]:
# PERCENTILE = 0
# PERCENTILE = 50
PERCENTILE = 75

In [6]:
mv_algorithm = MajorityVotingAlgorithm()
mv_generator = DataSetGenerator(algorithm)

In [7]:
parser = RectangleLabelElementParser()

### Select Scans for analysis

Then, select all Scans that you would like to analyse.

In [8]:
scans = models.Scan.query.all()
scans_ids = {scan.id for scan in scans}
print(f'Scan IDs: {scans_ids}')

Scan IDs: {'40761156-99db-4647-9b72-8e449e6cb54c', '03647f36-d874-480b-aaec-c2c3b9d80092', '7985ea16-d5a4-4bda-92db-43a3cc074216', 'b750320e-5664-4eff-b4c7-49e4f43ce6ca', '8b5d264d-bc2f-458f-ba68-2a987d55deae'}


### Select Label Elements for analysis

Now, select all Label Elements that should be analysed.

**IMPORTANT:** Currently Data Set Generator assumes only one Label Element per Slice. Don't worry, this will change in the near future.

In [9]:
query = models.RectangularLabelElement.query.join(models.Label).join(models.User)
query = query.filter(models.Label.scan_id.in_(scans_ids))
eti_rectangular_label_elements = query.filter(models.User.last_name == 'ETI').all()
gumed_rectangular_label_elements = query.filter(models.User.last_name == 'GUMED').all()
expert_rectangular_label_elements = query.filter(models.User.last_name == 'EXPERT').all()
combined_rectangular_label_elements = query.filter(models.User.last_name.in_(['ETI', 'GUMED'])).all()
print(f'There are {len(eti_rectangular_label_elements)} Rectangular Label Elements from ETI.')
print(f'There are {len(gumed_rectangular_label_elements)} Rectangular Label Elements from GUMED.')
print(f'There are {len(combined_rectangular_label_elements)} Rectangular Label Elements from ETI & GUMED.')
print(f'There are {len(expert_rectangular_label_elements)} Rectangular Label Elements from EXPERT.')
print(f'Example: {eti_rectangular_label_elements[0]}')

There are 1317 Rectangular Label Elements from ETI.
There are 494 Rectangular Label Elements from GUMED.
There are 1811 Rectangular Label Elements from ETI & GUMED.
There are 141 Rectangular Label Elements from EXPERT.
Example: <RectangularLabelElement: 8fb4c755-4a0e-4a03-8da2-72ba6593d9fb>


### Generate Ground Truth data set

Take your Label Elements and use generator to generate output Ground Truth annotations for each Slice that took part in the labeling process.

In [10]:
ground_truth_expert = mv_generator.generate(expert_rectangular_label_elements)
# ground_truth_expert

In [11]:
ground_truth_eti = generator.generate(eti_rectangular_label_elements)
# ground_truth_eti

In [12]:
combined_rectangular_label_elements = [s for s in combined_rectangular_label_elements
                                       if s.slice_index < len(s.label.scan.slices)]
ground_truth_combined = generator.generate(combined_rectangular_label_elements)
# ground_truth_combined

In [13]:
gumed_rectangular_label_elements = [s for s in gumed_rectangular_label_elements
                                    if s.slice_index < len(s.label.scan.slices)]
ground_truth_gumed = generator.generate(gumed_rectangular_label_elements)
# ground_truth_gumed

## [Table 1] Expert labels

In [14]:
labels_per_scan = {}
for slice_id in ground_truth_expert:
    scan_id = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first().id
    if ground_truth_expert[slice_id] is not None:
        if labels_per_scan.get(scan_id) is None:
            labels_per_scan[scan_id] = 0
        labels_per_scan[scan_id] += 1

In [15]:
slices_per_scan = {}
for scan in models.Scan.query.all():
    scan_id = scan.id
    slices_per_scan[scan_id] = len(scan.slices)

In [16]:
for scan_id in labels_per_scan:
    print(scan_id, '->', labels_per_scan[scan_id], 'labels,', slices_per_scan[scan_id], 'slices')

8b5d264d-bc2f-458f-ba68-2a987d55deae -> 10 labels, 28 slices
7985ea16-d5a4-4bda-92db-43a3cc074216 -> 29 labels, 50 slices
40761156-99db-4647-9b72-8e449e6cb54c -> 7 labels, 22 slices
b750320e-5664-4eff-b4c7-49e4f43ce6ca -> 10 labels, 28 slices
03647f36-d874-480b-aaec-c2c3b9d80092 -> 11 labels, 28 slices


## [Table 1] Valid vs. Invalid

In [17]:
# eti_valid_labels = {}
# eti_total_labels = {}

# for e in eti_rectangular_label_elements:
#     scan_id = e.label.scan.id
#     slice_id = e.label.scan.slices[e.slice_index].id
#     if eti_valid_labels.get(scan_id) is None:
#         eti_valid_labels[scan_id] = 0
#         eti_total_labels[scan_id] = 0
#     if ground_truth_expert[slice_id] is not None:
#         eti_valid_labels[scan_id] += 1
#     eti_total_labels[scan_id] += 1

# gumed_valid_labels = {}
# gumed_total_labels = {}

# for e in gumed_rectangular_label_elements:
#     if e.slice_index >= len(e.label.scan.slices):
#         continue
#     scan_id = e.label.scan.id
#     slice_id = e.label.scan.slices[e.slice_index].id
#     if gumed_valid_labels.get(scan_id) is None:
#         gumed_valid_labels[scan_id] = 0
#         gumed_total_labels[scan_id] = 0
#     if ground_truth_expert[slice_id] is not None:
#         gumed_valid_labels[scan_id] += 1
#     gumed_total_labels[scan_id] += 1

# for scan_id in SCAN_IDS:
#     print(' {:.3f}%'.format(100 * eti_valid_labels[scan_id] / eti_total_labels[scan_id]),
#           '\t{:.3f}%'.format(100 * (eti_total_labels[scan_id] - eti_valid_labels[scan_id]) / eti_total_labels[scan_id]),
#           '\t{:.3f}%'.format(100 * gumed_valid_labels[scan_id] / gumed_total_labels[scan_id]),
#           '\t{:.3f}%'.format(100 * (gumed_total_labels[scan_id] - gumed_valid_labels[scan_id]) / gumed_total_labels[scan_id]))

## [Table 2] Dispersion

In [18]:
# eti_x_differences = collections.defaultdict(lambda: [])
# eti_y_differences = collections.defaultdict(lambda: [])
# eti_w_differences = collections.defaultdict(lambda: [])
# eti_h_differences = collections.defaultdict(lambda: [])

# for e in eti_rectangular_label_elements:
#     scan_id = e.label.scan.id
#     slice_id = e.label.scan.slices[e.slice_index].id
#     gt_rect = ground_truth_expert.get(slice_id)
#     gt_rect = gt_rect if gt_rect is not None else np.array((0,0,0,0))
#     eti_x_differences[scan_id].append(abs(e.x - gt_rect[0]))
#     eti_y_differences[scan_id].append(abs(e.y - gt_rect[1]))
#     eti_w_differences[scan_id].append(abs(e.width - (gt_rect[2] - gt_rect[0])))
#     eti_h_differences[scan_id].append(abs(e.height - (gt_rect[3] - gt_rect[1])))

# gumed_x_differences = collections.defaultdict(lambda: [])
# gumed_y_differences = collections.defaultdict(lambda: [])
# gumed_w_differences = collections.defaultdict(lambda: [])
# gumed_h_differences = collections.defaultdict(lambda: [])

# for e in gumed_rectangular_label_elements:
#     scan_id = e.label.scan.id
#     slice_id = e.label.scan.slices[e.slice_index].id
#     gt_rect = ground_truth_expert.get(slice_id)
#     gt_rect = gt_rect if gt_rect is not None else np.array((0,0,0,0))
#     gumed_x_differences[scan_id].append(abs(e.x - gt_rect[0]))
#     gumed_y_differences[scan_id].append(abs(e.y - gt_rect[1]))
#     gumed_w_differences[scan_id].append(abs(e.width - (gt_rect[2] - gt_rect[0])))
#     gumed_h_differences[scan_id].append(abs(e.height - (gt_rect[3] - gt_rect[1])))
    
# for scan_id in SCAN_IDS:
#     print(' {:.3f}%'.format(np.std(eti_x_differences[scan_id]) * 100),
#           '\t{:.3f}%'.format(np.std(eti_y_differences[scan_id]) * 100),
#           '\t{:.3f}%'.format(np.std(eti_w_differences[scan_id]) * 100),
#           '\t{:.3f}%'.format(np.std(eti_h_differences[scan_id]) * 100),
#           '\t{:.3f}%'.format(np.std(gumed_x_differences[scan_id]) * 100),
#           '\t{:.3f}%'.format(np.std(gumed_y_differences[scan_id]) * 100),
#           '\t{:.3f}%'.format(np.std(gumed_w_differences[scan_id]) * 100),
#           '\t{:.3f}%'.format(np.std(gumed_h_differences[scan_id]) * 100))

## [Table 3] Dispersion in generated Ground Truth

In [19]:
# gt_eti_x_differences = collections.defaultdict(lambda: 0)
# gt_eti_y_differences = collections.defaultdict(lambda: 0)
# gt_eti_w_differences = collections.defaultdict(lambda: 0)
# gt_eti_h_differences = collections.defaultdict(lambda: 0)
# gt_eti_total = collections.defaultdict(lambda: 0)

# gt_gumed_x_differences = collections.defaultdict(lambda: 0)
# gt_gumed_y_differences = collections.defaultdict(lambda: 0)
# gt_gumed_w_differences = collections.defaultdict(lambda: 0)
# gt_gumed_h_differences = collections.defaultdict(lambda: 0)
# gt_gumed_total = collections.defaultdict(lambda: 0)

# for slice_id in ground_truth_eti:
#     scan_id = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first().id
#     gt_eti = ground_truth_eti.get(slice_id)
#     gt_eti = gt_eti if gt_eti is not None else np.array((0,0,0,0))
#     gt_rect = ground_truth_expert.get(slice_id)
#     gt_rect = gt_rect if gt_rect is not None else np.array((0,0,0,0))
#     gt_eti_x_differences[scan_id] += abs(gt_eti[0] - gt_rect[0])
#     gt_eti_y_differences[scan_id] += abs(gt_eti[1] - gt_rect[1])
#     gt_eti_w_differences[scan_id] += abs((gt_eti[2] - gt_eti[0]) - (gt_rect[2] - gt_rect[0]))
#     gt_eti_h_differences[scan_id] += abs((gt_eti[3] - gt_eti[1]) - (gt_rect[3] - gt_rect[1]))
#     gt_eti_total[scan_id] += 1

# for slice_id in ground_truth_gumed:
#     scan_id = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first().id
#     gt_gumed = ground_truth_gumed.get(slice_id)
#     gt_gumed = gt_gumed if gt_gumed is not None else np.array((0,0,0,0))
#     gt_rect = ground_truth_expert.get(slice_id)
#     gt_rect = gt_rect if gt_rect is not None else np.array((0,0,0,0))
#     gt_gumed_x_differences[scan_id] += abs(gt_gumed[0] - gt_rect[0])
#     gt_gumed_y_differences[scan_id] += abs(gt_gumed[1] - gt_rect[1])
#     gt_gumed_w_differences[scan_id] += abs((gt_gumed[2] - gt_gumed[0]) - (gt_rect[2] - gt_rect[0]))
#     gt_gumed_h_differences[scan_id] += abs((gt_gumed[3] - gt_gumed[1]) - (gt_rect[3] - gt_rect[1]))
#     gt_gumed_total[scan_id] += 1

# for scan_id in SCAN_IDS:
#     print(' {:.3f}%'.format(gt_eti_x_differences[scan_id] / gt_eti_total[scan_id] * 100),
#           '\t{:.3f}%'.format(gt_eti_y_differences[scan_id] / gt_eti_total[scan_id] * 100),
#           '\t{:.3f}%'.format(gt_eti_w_differences[scan_id] / gt_eti_total[scan_id] * 100),
#           '\t{:.3f}%'.format(gt_eti_h_differences[scan_id] / gt_eti_total[scan_id] * 100),
#           '\t{:.3f}%'.format(gt_gumed_x_differences[scan_id] / gt_gumed_total[scan_id] * 100),
#           '\t{:.3f}%'.format(gt_gumed_y_differences[scan_id] / gt_gumed_total[scan_id] * 100),
#           '\t{:.3f}%'.format(gt_gumed_w_differences[scan_id] / gt_gumed_total[scan_id] * 100),
#           '\t{:.3f}%'.format(gt_gumed_h_differences[scan_id] / gt_gumed_total[scan_id] * 100))

### [GUMED] Check Users' Specificity and Sensitivity

MedTagger supports easy calculation of Users' Specificity, Sensitivity and Score based on their annotations and Ground Truth data set.

In [20]:
gumed_users = set(element.label.owner for element in gumed_rectangular_label_elements)
print(f'There are {len(gumed_users)} Users.')

There are 13 Users.


In [21]:
gumed_users_specificity, gumed_users_sensitivity, gumed_users_scores = \
    compute_specificity_and_sensitivity_for_users(algorithm, gumed_users,
                                                  gumed_rectangular_label_elements,
                                                  ground_truth_gumed)

In [22]:
users_ids = set(u.id for u in gumed_users)
for user_id in users_ids:
    print(f'User #{user_id:03d}: Sensitivity={gumed_users_sensitivity[user_id]:1.5f} '
          f'Specificity={gumed_users_specificity[user_id]:1.5f} Score={gumed_users_scores[user_id]:1.5f}')

User #102: Sensitivity=0.01370 Specificity=1.00000 Score=0.00019
User #103: Sensitivity=0.83562 Specificity=1.00000 Score=0.69825
User #104: Sensitivity=0.02740 Specificity=0.98795 Score=0.00024
User #106: Sensitivity=0.13699 Specificity=0.98795 Score=0.01561
User #107: Sensitivity=0.95890 Specificity=0.97590 Score=0.87387
User #109: Sensitivity=0.13699 Specificity=1.00000 Score=0.01877
User #116: Sensitivity=0.80822 Specificity=1.00000 Score=0.65322
User #117: Sensitivity=0.15068 Specificity=0.98795 Score=0.01922
User #118: Sensitivity=0.65753 Specificity=1.00000 Score=0.43235
User #119: Sensitivity=0.05479 Specificity=0.98795 Score=0.00183
User #120: Sensitivity=0.98630 Specificity=0.97590 Score=0.92584
User #121: Sensitivity=0.56164 Specificity=0.98795 Score=0.30206
User #123: Sensitivity=0.94521 Specificity=0.96386 Score=0.82639


In [23]:
# figures.specificity_vs_sensitivity(gumed_users_specificity, gumed_users_sensitivity)

### [GUMED] Compare Labeling Time to Users' Score

You can also check how User's mean Labeling Time outputs in their achieved scores.

In [24]:
# figures.mean_labeling_time_vs_score(gumed_rectangular_label_elements, gumed_users_scores)

### [ETI] Check Users' Specificity and Sensitivity

MedTagger supports easy calculation of Users' Specificity, Sensitivity and Score based on their annotations and Ground Truth data set.

In [25]:
eti_users = set(element.label.owner for element in eti_rectangular_label_elements)
print(f'There are {len(eti_users)} Users.')

There are 35 Users.


In [26]:
eti_users_specificity, eti_users_sensitivity, eti_users_scores = \
    compute_specificity_and_sensitivity_for_users(algorithm, eti_users,
                                                  eti_rectangular_label_elements,
                                                  ground_truth_eti)

In [27]:
users_ids = set(u.id for u in eti_users)
for user_id in users_ids:
    print(f'User #{user_id:03d}: Sensitivity={eti_users_sensitivity[user_id]:1.5f} '
          f'Specificity={eti_users_specificity[user_id]:1.5f} Score={eti_users_scores[user_id]:1.5f}')

User #003: Sensitivity=0.47561 Specificity=1.00000 Score=0.22620
User #005: Sensitivity=0.48780 Specificity=1.00000 Score=0.23795
User #025: Sensitivity=0.50000 Specificity=1.00000 Score=0.25000
User #031: Sensitivity=0.48780 Specificity=0.98649 Score=0.22495
User #033: Sensitivity=0.24390 Specificity=1.00000 Score=0.05949
User #034: Sensitivity=0.45122 Specificity=1.00000 Score=0.20360
User #035: Sensitivity=0.40244 Specificity=1.00000 Score=0.16196
User #036: Sensitivity=0.39024 Specificity=1.00000 Score=0.15229
User #037: Sensitivity=0.47561 Specificity=1.00000 Score=0.22620
User #039: Sensitivity=0.01220 Specificity=0.97297 Score=0.00022
User #040: Sensitivity=0.47561 Specificity=1.00000 Score=0.22620
User #041: Sensitivity=0.45122 Specificity=0.98649 Score=0.19159
User #042: Sensitivity=0.46341 Specificity=1.00000 Score=0.21475
User #043: Sensitivity=0.39024 Specificity=1.00000 Score=0.15229
User #044: Sensitivity=0.90244 Specificity=1.00000 Score=0.81440
User #048: Sensitivity=0.

In [28]:
# figures.specificity_vs_sensitivity(eti_users_specificity, eti_users_sensitivity)

### [ETI] Compare Labeling Time to Users' Score

You can also check how User's mean Labeling Time outputs in their achieved scores.

In [29]:
# figures.mean_labeling_time_vs_score(eti_rectangular_label_elements, eti_users_scores)

### [COMBINED] Check Users' Specificity and Sensitivity

MedTagger supports easy calculation of Users' Specificity, Sensitivity and Score based on their annotations and Ground Truth data set.

In [30]:
combined_users = set(element.label.owner for element in combined_rectangular_label_elements)
print(f'There are {len(combined_users)} Users.')

There are 48 Users.


In [31]:
combined_users_specificity, combined_users_sensitivity, combined_users_scores = \
    compute_specificity_and_sensitivity_for_users(algorithm, combined_users,
                                                  combined_rectangular_label_elements,
                                                  ground_truth_combined)

In [32]:
users_ids = set(u.id for u in combined_users)
for user_id in users_ids:
    print(f'User #{user_id:03d}: Sensitivity={combined_users_sensitivity[user_id]:1.5f} '
          f'Specificity={combined_users_specificity[user_id]:1.5f} Score={combined_users_scores[user_id]:1.5f}')

User #003: Sensitivity=0.44828 Specificity=1.00000 Score=0.20095
User #005: Sensitivity=0.45977 Specificity=1.00000 Score=0.21139
User #025: Sensitivity=0.49425 Specificity=1.00000 Score=0.24429
User #031: Sensitivity=0.45977 Specificity=0.98551 Score=0.19827
User #033: Sensitivity=0.22989 Specificity=1.00000 Score=0.05285
User #034: Sensitivity=0.42529 Specificity=1.00000 Score=0.18087
User #035: Sensitivity=0.37931 Specificity=1.00000 Score=0.14388
User #036: Sensitivity=0.36782 Specificity=1.00000 Score=0.13529
User #037: Sensitivity=0.44828 Specificity=1.00000 Score=0.20095
User #039: Sensitivity=0.01149 Specificity=0.97101 Score=0.00031
User #040: Sensitivity=0.44828 Specificity=1.00000 Score=0.20095
User #041: Sensitivity=0.42529 Specificity=1.00000 Score=0.18087
User #042: Sensitivity=0.43678 Specificity=1.00000 Score=0.19078
User #043: Sensitivity=0.36782 Specificity=1.00000 Score=0.13529
User #044: Sensitivity=0.85057 Specificity=1.00000 Score=0.72348
User #048: Sensitivity=0.

In [33]:
# figures.specificity_vs_sensitivity(combined_users_specificity, combined_users_sensitivity)

### [COMBINED] Compare Labeling Time to Users' Score

You can also check how User's mean Labeling Time outputs in their achieved scores.

In [34]:
# figures.mean_labeling_time_vs_score(eti_rectangular_label_elements, eti_users_scores)

## [GUMED] Only best labels

In [35]:
gumed_raw_scores = np.array(list(gumed_users_scores.values()))
gumed_median_score = np.percentile(gumed_raw_scores, PERCENTILE)
best_gumed_users_id = [u_id for u_id in gumed_users_scores if gumed_users_scores[u_id] >= gumed_median_score]
best_gumed_users_id

[103, 107, 120, 123]

In [36]:
best_gumed_rectangular_label_elements = [e for e in gumed_rectangular_label_elements
                                         if e.label.owner_id in best_gumed_users_id]
ground_truth_gumed_best = generator.generate(best_gumed_rectangular_label_elements)
# ground_truth_gumed_best

## [ETI] Only best labels

In [37]:
eti_raw_scores = np.array(list(eti_users_scores.values()))
eti_median_score = np.percentile(eti_raw_scores, PERCENTILE)
best_eti_users_id = [u_id for u_id in eti_users_scores if eti_users_scores[u_id] >= eti_median_score]
# best_eti_users_id

In [38]:
best_eti_rectangular_label_elements = [e for e in eti_rectangular_label_elements
                                       if e.label.owner_id in best_eti_users_id]
ground_truth_eti_best = generator.generate(best_eti_rectangular_label_elements)
# ground_truth_eti_best

## [COMBINED] Only best labels

In [39]:
combined_raw_scores = np.array(list(combined_users_scores.values()))
combined_median_score = np.percentile(combined_raw_scores, PERCENTILE)
best_combined_users_id = [u_id for u_id in combined_users_scores
                          if combined_users_scores[u_id] >= combined_median_score]
# best_combined_users_id

In [40]:
best_combined_rectangular_label_elements = [e for e in combined_rectangular_label_elements
                                            if e.label.owner_id in best_combined_users_id]
ground_truth_combined_best = generator.generate(best_combined_rectangular_label_elements)
# ground_truth_combined_best

## [Table 1]

In [41]:
# gumed_valid_labels = {}
# gumed_total_labels = {}

# for e in best_gumed_rectangular_label_elements:
#     scan_id = e.label.scan.id
#     slice_id = e.label.scan.slices[e.slice_index].id
#     if gumed_valid_labels.get(scan_id) is None:
#         gumed_valid_labels[scan_id] = 0
#         gumed_total_labels[scan_id] = 0
#     if ground_truth_expert[slice_id] is not None:
#         gumed_valid_labels[scan_id] += 1
#     gumed_total_labels[scan_id] += 1

# eti_valid_labels = {}
# eti_total_labels = {}

# for e in best_eti_rectangular_label_elements:
#     scan_id = e.label.scan.id
#     slice_id = e.label.scan.slices[e.slice_index].id
#     if eti_valid_labels.get(scan_id) is None:
#         eti_valid_labels[scan_id] = 0
#         eti_total_labels[scan_id] = 0
#     if ground_truth_expert[slice_id] is not None:
#         eti_valid_labels[scan_id] += 1
#     eti_total_labels[scan_id] += 1

# for scan_id in SCAN_IDS:
#     print(' {:.3f}%'.format(100 * eti_valid_labels[scan_id] / eti_total_labels[scan_id]),
#           '\t{:.3f}%'.format(100 * (eti_total_labels[scan_id] - eti_valid_labels[scan_id]) / eti_total_labels[scan_id]),
#           '\t{:.3f}%'.format(100 * gumed_valid_labels[scan_id] / gumed_total_labels[scan_id]),
#           '\t{:.3f}%'.format(100 * (gumed_total_labels[scan_id] - gumed_valid_labels[scan_id]) / gumed_total_labels[scan_id]))

## [Table 2.1] Dispersion

In [42]:
# gumed_x_differences = collections.defaultdict(lambda: [])
# gumed_y_differences = collections.defaultdict(lambda: [])
# gumed_w_differences = collections.defaultdict(lambda: [])
# gumed_h_differences = collections.defaultdict(lambda: [])

# for e in best_gumed_rectangular_label_elements:
#     scan_id = e.label.scan.id
#     slice_id = e.label.scan.slices[e.slice_index].id
#     gt_rect = ground_truth_expert.get(slice_id)
#     gt_rect = gt_rect if gt_rect is not None else np.array((0,0,0,0))
#     gumed_x_differences[scan_id].append(abs(e.x - gt_rect[0]))
#     gumed_y_differences[scan_id].append(abs(e.y - gt_rect[1]))
#     gumed_w_differences[scan_id].append(abs(e.width - (gt_rect[2] - gt_rect[0])))
#     gumed_h_differences[scan_id].append(abs(e.height - (gt_rect[3] - gt_rect[1])))

# eti_x_differences = collections.defaultdict(lambda: [])
# eti_y_differences = collections.defaultdict(lambda: [])
# eti_w_differences = collections.defaultdict(lambda: [])
# eti_h_differences = collections.defaultdict(lambda: [])

# for e in best_eti_rectangular_label_elements:
#     scan_id = e.label.scan.id
#     slice_id = e.label.scan.slices[e.slice_index].id
#     gt_rect = ground_truth_expert.get(slice_id)
#     gt_rect = gt_rect if gt_rect is not None else np.array((0,0,0,0))
#     eti_x_differences[scan_id].append(abs(e.x - gt_rect[0]))
#     eti_y_differences[scan_id].append(abs(e.y - gt_rect[1]))
#     eti_w_differences[scan_id].append(abs(e.width - (gt_rect[2] - gt_rect[0])))
#     eti_h_differences[scan_id].append(abs(e.height - (gt_rect[3] - gt_rect[1])))

# for scan_id in SCAN_IDS:
#     print(' {:.3f}%'.format(np.std(eti_x_differences[scan_id]) * 100),
#           '\t{:.3f}%'.format(np.std(eti_y_differences[scan_id]) * 100),
#           '\t{:.3f}%'.format(np.std(eti_w_differences[scan_id]) * 100),
#           '\t{:.3f}%'.format(np.std(eti_h_differences[scan_id]) * 100),
#           '\t{:.3f}%'.format(np.std(gumed_x_differences[scan_id]) * 100),
#           '\t{:.3f}%'.format(np.std(gumed_y_differences[scan_id]) * 100),
#           '\t{:.3f}%'.format(np.std(gumed_w_differences[scan_id]) * 100),
#           '\t{:.3f}%'.format(np.std(gumed_h_differences[scan_id]) * 100))

## [Table 3.1] Dispersion on best people

In [43]:
# gt_eti_x_differences = collections.defaultdict(lambda: 0)
# gt_eti_y_differences = collections.defaultdict(lambda: 0)
# gt_eti_w_differences = collections.defaultdict(lambda: 0)
# gt_eti_h_differences = collections.defaultdict(lambda: 0)
# gt_eti_total = collections.defaultdict(lambda: 0)

# gt_gumed_x_differences = collections.defaultdict(lambda: 0)
# gt_gumed_y_differences = collections.defaultdict(lambda: 0)
# gt_gumed_w_differences = collections.defaultdict(lambda: 0)
# gt_gumed_h_differences = collections.defaultdict(lambda: 0)
# gt_gumed_total = collections.defaultdict(lambda: 0)

# for slice_id in ground_truth_eti_best:
#     scan_id = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first().id
#     gt_eti = ground_truth_eti_best.get(slice_id)
#     gt_eti = gt_eti if gt_eti is not None else np.array((0,0,0,0))
#     gt_rect = ground_truth_expert.get(slice_id)
#     gt_rect = gt_rect if gt_rect is not None else np.array((0,0,0,0))
#     gt_eti_x_differences[scan_id] += abs(gt_eti[0] - gt_rect[0])
#     gt_eti_y_differences[scan_id] += abs(gt_eti[1] - gt_rect[1])
#     gt_eti_w_differences[scan_id] += abs((gt_eti[2] - gt_eti[0]) - (gt_rect[2] - gt_rect[0]))
#     gt_eti_h_differences[scan_id] += abs((gt_eti[3] - gt_eti[1]) - (gt_rect[3] - gt_rect[1]))
#     gt_eti_total[scan_id] += 1

# for slice_id in ground_truth_gumed_best:
#     scan_id = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first().id
#     gt_gumed = ground_truth_gumed_best.get(slice_id)
#     gt_gumed = gt_gumed if gt_gumed is not None else np.array((0,0,0,0))
#     gt_rect = ground_truth_expert.get(slice_id)
#     gt_rect = gt_rect if gt_rect is not None else np.array((0,0,0,0))
#     gt_gumed_x_differences[scan_id] += abs(gt_gumed[0] - gt_rect[0])
#     gt_gumed_y_differences[scan_id] += abs(gt_gumed[1] - gt_rect[1])
#     gt_gumed_w_differences[scan_id] += abs((gt_gumed[2] - gt_gumed[0]) - (gt_rect[2] - gt_rect[0]))
#     gt_gumed_h_differences[scan_id] += abs((gt_gumed[3] - gt_gumed[1]) - (gt_rect[3] - gt_rect[1]))
#     gt_gumed_total[scan_id] += 1

# for scan_id in SCAN_IDS:
#     print(' {:.3f}%'.format(gt_eti_x_differences[scan_id] / gt_eti_total[scan_id] * 100),
#           '\t{:.3f}%'.format(gt_eti_y_differences[scan_id] / gt_eti_total[scan_id] * 100),
#           '\t{:.3f}%'.format(gt_eti_w_differences[scan_id] / gt_eti_total[scan_id] * 100),
#           '\t{:.3f}%'.format(gt_eti_h_differences[scan_id] / gt_eti_total[scan_id] * 100),
#           '\t{:.3f}%'.format(gt_gumed_x_differences[scan_id] / gt_gumed_total[scan_id] * 100),
#           '\t{:.3f}%'.format(gt_gumed_y_differences[scan_id] / gt_gumed_total[scan_id] * 100),
#           '\t{:.3f}%'.format(gt_gumed_w_differences[scan_id] / gt_gumed_total[scan_id] * 100),
#           '\t{:.3f}%'.format(gt_gumed_h_differences[scan_id] / gt_gumed_total[scan_id] * 100))

In [44]:
# #
# # DLA DAWIDA
# #
# user_emails = set()

# gumed_differences = collections.defaultdict(lambda: collections.defaultdict(lambda: 0))
# gumed_total = collections.defaultdict(lambda: collections.defaultdict(lambda: 0))

# for e in best_gumed_rectangular_label_elements:
#     user_id = e.label.owner.email
#     user_emails.add(user_id)
#     scan_id = e.label.scan.id
#     slice_id = e.label.scan.slices[e.slice_index].id
#     gt_rect = ground_truth_expert.get(slice_id)
#     gt_rect = gt_rect if gt_rect is not None else np.array((0,0,0,0))
#     gumed_rect = parser.convert_to_numpy([e])[0]
    
#     iou = parser.compute_intersection_over_union(gumed_rect, gt_rect)
#     if not np.isnan(iou):
#         gumed_differences[user_id][scan_id] += iou
#         gumed_total[user_id][scan_id] += 1

# for slice_id in ground_truth_expert:
#     for gumed_id in best_gumed_users_id:
#         user = models.User.query.filter(models.User.id == gumed_id).first()
        
#         gumed_label_element = next((e for e in best_gumed_rectangular_label_elements
#                                     if e.label.scan.slices[e.slice_index].id == slice_id
#                                       and e.label.owner_id == gumed_id
#                                    ), None)
        
#         if ground_truth_expert.get(slice_id) is not None and gumed_label_element is None:
#             scan_id = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first().id
#             gumed_differences[user.email][scan_id] += 0
#             gumed_total[user.email][scan_id] += 1

# eti_differences = collections.defaultdict(lambda: collections.defaultdict(lambda: 0))
# eti_total = collections.defaultdict(lambda: collections.defaultdict(lambda: 0))

# for e in best_eti_rectangular_label_elements:
#     user_id = e.label.owner.email
#     user_emails.add(user_id)
#     scan_id = e.label.scan.id
#     slice_id = e.label.scan.slices[e.slice_index].id
#     gt_rect = ground_truth_expert.get(slice_id)
#     gt_rect = gt_rect if gt_rect is not None else np.array((0,0,0,0))
#     eti_rect = parser.convert_to_numpy([e])[0]
    
#     iou = parser.compute_intersection_over_union(eti_rect, gt_rect)
#     if not np.isnan(iou):
#         eti_differences[user_id][scan_id] += iou
#         eti_total[user_id][scan_id] += 1


# for slice_id in ground_truth_expert:
#     for eti_id in best_eti_users_id:
#         user = models.User.query.filter(models.User.id == eti_id).first()
        
#         eti_label_element = next((e for e in best_eti_rectangular_label_elements
#                                     if e.label.scan.slices[e.slice_index].id == slice_id
#                                       and e.label.owner_id == eti_id
#                                    ), None)
        
#         if ground_truth_expert.get(slice_id) is not None and eti_label_element is None:
#             scan_id = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first().id
#             eti_differences[user.email][scan_id] += 0
#             eti_total[user.email][scan_id] += 1
        
# for user_id in user_emails:
#     for scan_id in SCAN_IDS:
#         if eti_total[user_id][scan_id] > 0:
#             iou = eti_differences[user_id][scan_id] / eti_total[user_id][scan_id] * 100
#         elif gumed_total[user_id][scan_id] > 0:
#             iou = gumed_differences[user_id][scan_id] / gumed_total[user_id][scan_id] * 100
#         else:
#             iou = 0
#         label = models.Label.query.join(models.User).filter(models.User.email == user_id)\
#                    .filter(models.Label.scan_id == scan_id).first()
#         cant_see_anything = False if label is None else label.comment == 'This is an empty Label'
#         timestamp = 0 if label is None else label.labeling_time
#         print('{}\t{}\t{:.3f}%\t{}\t{}'.format(user_id, scan_id, iou, timestamp, cant_see_anything))

## Intersection over Union - między wprowadzonymi etykietami a prawdziwym Ground Truth

In [45]:
gumed_differences = collections.defaultdict(lambda: 0)
gumed_total = collections.defaultdict(lambda: 0)

for e in best_gumed_rectangular_label_elements:
    scan_id = e.label.scan.id
    slice_id = e.label.scan.slices[e.slice_index].id
    gt_rect = ground_truth_expert.get(slice_id)
    gt_rect = gt_rect if gt_rect is not None else np.array((0,0,0,0))
    gumed_rect = parser.convert_to_numpy([e])[0]
    
    iou = parser.compute_intersection_over_union(gumed_rect, gt_rect)
    if not np.isnan(iou):
        gumed_differences[scan_id] += iou
        gumed_total[scan_id] += 1

for slice_id in ground_truth_expert:
    for gumed_id in best_gumed_users_id:
        user = models.User.query.filter(models.User.id == gumed_id).first()
        
        gumed_label_element = next((e for e in best_gumed_rectangular_label_elements
                                    if e.label.scan.slices[e.slice_index].id == slice_id
                                      and e.label.owner_id == gumed_id
                                   ), None)
        
        if ground_truth_expert.get(slice_id) is not None and gumed_label_element is None:
            scan_id = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first().id
            gumed_differences[scan_id] += 0
            gumed_total[scan_id] += 1
        
eti_differences = collections.defaultdict(lambda: 0)
eti_total = collections.defaultdict(lambda: 0)

for e in best_eti_rectangular_label_elements:
    scan_id = e.label.scan.id
    slice_id = e.label.scan.slices[e.slice_index].id
    gt_rect = ground_truth_expert.get(slice_id)
    gt_rect = gt_rect if gt_rect is not None else np.array((0,0,0,0))
    eti_rect = parser.convert_to_numpy([e])[0]
    
    iou = parser.compute_intersection_over_union(eti_rect, gt_rect)
    if not np.isnan(iou):
        eti_differences[scan_id] += iou
        eti_total[scan_id] += 1

for slice_id in ground_truth_expert:
    for eti_id in best_eti_users_id:
        user = models.User.query.filter(models.User.id == eti_id).first()
        
        eti_label_element = next((e for e in best_eti_rectangular_label_elements
                                    if e.label.scan.slices[e.slice_index].id == slice_id
                                      and e.label.owner_id == eti_id
                                   ), None)
        
        if ground_truth_expert.get(slice_id) is not None and eti_label_element is None:
            scan_id = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first().id
            eti_differences[scan_id] += 0
            eti_total[scan_id] += 1
        
combined_differences = collections.defaultdict(lambda: 0)
combined_total = collections.defaultdict(lambda: 0)

for e in best_combined_rectangular_label_elements:
    scan_id = e.label.scan.id
    slice_id = e.label.scan.slices[e.slice_index].id
    gt_rect = ground_truth_expert.get(slice_id)
    gt_rect = gt_rect if gt_rect is not None else np.array((0,0,0,0))
    combined_rect = parser.convert_to_numpy([e])[0]
    
    iou = parser.compute_intersection_over_union(combined_rect, gt_rect)
    if not np.isnan(iou):
        combined_differences[scan_id] += iou
        combined_total[scan_id] += 1

for slice_id in ground_truth_expert:
    for eti_id in best_combined_users_id:
        user = models.User.query.filter(models.User.id == eti_id).first()
        
        combined_label_element = next((e for e in best_combined_rectangular_label_elements
                                    if e.label.scan.slices[e.slice_index].id == slice_id
                                      and e.label.owner_id == eti_id
                                   ), None)
        
        if ground_truth_expert.get(slice_id) is not None and combined_label_element is None:
            scan_id = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first().id
            combined_differences[scan_id] += 0
            combined_total[scan_id] += 1

for scan_id in SCAN_IDS:
    print(' {:.3f}%'.format(eti_differences[scan_id] / eti_total[scan_id] * 100),
          '\t{:.3f}%'.format(gumed_differences[scan_id] / gumed_total[scan_id] * 100),
          '\t{:.3f}%'.format(combined_differences[scan_id] / combined_total[scan_id] * 100))

 78.654% 	77.347% 	75.461%
 74.215% 	70.247% 	71.912%
 66.708% 	69.971% 	63.470%
 65.700% 	74.465% 	64.351%
 32.766% 	67.650% 	50.358%


## Intersection over Union - między wyliczonym Ground Truth a prawdziwym Ground Truth

In [46]:
gt_eti_differences = collections.defaultdict(lambda: 0)
gt_eti_total = collections.defaultdict(lambda: 0)

for slice_id in ground_truth_eti_best:
    scan_id = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first().id
    gt_eti = ground_truth_eti_best.get(slice_id)
    gt_eti = gt_eti if gt_eti is not None else np.array((0,0,0,0))
    gt_rect = ground_truth_expert.get(slice_id)
    gt_rect = gt_rect if gt_rect is not None else np.array((0,0,0,0))

    iou = parser.compute_intersection_over_union(gt_eti, gt_rect)
    if not np.isnan(iou):
        gt_eti_differences[scan_id] += iou
        gt_eti_total[scan_id] += 1

gt_gumed_differences = collections.defaultdict(lambda: 0)
gt_gumed_total = collections.defaultdict(lambda: 0)
    
for slice_id in ground_truth_gumed_best:
    scan_id = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first().id
    gt_gumed = ground_truth_gumed_best.get(slice_id)
    gt_gumed = gt_gumed if gt_gumed is not None else np.array((0,0,0,0))
    gt_rect = ground_truth_expert.get(slice_id)
    gt_rect = gt_rect if gt_rect is not None else np.array((0,0,0,0))
    
    iou = parser.compute_intersection_over_union(gt_gumed, gt_rect)
    if not np.isnan(iou):
        gt_gumed_differences[scan_id] += iou
        gt_gumed_total[scan_id] += 1

gt_combined_differences = collections.defaultdict(lambda: 0)
gt_combined_total = collections.defaultdict(lambda: 0)
    
for slice_id in ground_truth_combined_best:
    scan_id = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first().id
    gt_combined = ground_truth_combined_best.get(slice_id)
    gt_combined = gt_combined if gt_combined is not None else np.array((0,0,0,0))
    gt_rect = ground_truth_expert.get(slice_id)
    gt_rect = gt_rect if gt_rect is not None else np.array((0,0,0,0))
    
    iou = parser.compute_intersection_over_union(gt_combined, gt_rect)
    if not np.isnan(iou):
        gt_combined_differences[scan_id] += iou
        gt_combined_total[scan_id] += 1

for scan_id in SCAN_IDS:
    print(' {:.3f}%'.format(gt_eti_differences[scan_id] / gt_eti_total[scan_id] * 100),
          '\t{:.3f}%'.format(gt_gumed_differences[scan_id] / gt_gumed_total[scan_id] * 100),
          '\t{:.3f}%'.format(gt_combined_differences[scan_id] / gt_combined_total[scan_id] * 100))

/Users/jakubpowierza/Projects/MedTagger/backend/medtagger/ground_truth/parsers/rectangle.py:49: RuntimeWarning: invalid value encountered in long_scalars
  return intersection / union


 80.245% 	87.511% 	79.155%
 77.615% 	74.112% 	69.559%
 71.506% 	78.722% 	69.687%
 70.207% 	81.526% 	69.165%
 71.232% 	72.093% 	71.115%


## Mean Average Precision @ 0.75 - między wprowadzonymi etykietami a prawdziwym Ground Truth

In [47]:
THRESHOLD = 0.75

gumed_differences = collections.defaultdict(lambda: 0)
gumed_total = collections.defaultdict(lambda: 0)

for e in best_gumed_rectangular_label_elements:
    scan_id = e.label.scan.id
    slice_id = e.label.scan.slices[e.slice_index].id
    gt_rect = ground_truth_expert.get(slice_id)
    gt_rect = gt_rect if gt_rect is not None else np.array((0,0,0,0))
    gumed_rect = parser.convert_to_numpy([e])[0]
    
    iou = parser.compute_intersection_over_union(gumed_rect, gt_rect)
    iou = 1.0 if np.isnan(iou) else iou
    gumed_differences[scan_id] += 1 if iou >= THRESHOLD else 0
    gumed_total[scan_id] += 1

for slice_id in ground_truth_expert:
    for gumed_id in best_gumed_users_id:
        user = models.User.query.filter(models.User.id == gumed_id).first()
        
        gumed_label_element = next((e for e in best_gumed_rectangular_label_elements
                                    if e.label.scan.slices[e.slice_index].id == slice_id
                                      and e.label.owner_id == gumed_id
                                   ), None)
        
        if ground_truth_expert.get(slice_id) is not None and gumed_label_element is None:
            scan_id = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first().id
            gumed_differences[scan_id] += 0
            gumed_total[scan_id] += 1
    
eti_differences = collections.defaultdict(lambda: 0)
eti_total = collections.defaultdict(lambda: 0)

for e in best_eti_rectangular_label_elements:
    scan_id = e.label.scan.id
    slice_id = e.label.scan.slices[e.slice_index].id
    gt_rect = ground_truth_expert.get(slice_id)
    gt_rect = gt_rect if gt_rect is not None else np.array((0,0,0,0))
    eti_rect = parser.convert_to_numpy([e])[0]
    
    iou = parser.compute_intersection_over_union(eti_rect, gt_rect)
    iou = 1.0 if np.isnan(iou) else iou
    eti_differences[scan_id] += 1 if iou >= THRESHOLD else 0
    eti_total[scan_id] += 1

for slice_id in ground_truth_expert:
    for eti_id in best_eti_users_id:
        user = models.User.query.filter(models.User.id == eti_id).first()
        
        eti_label_element = next((e for e in best_eti_rectangular_label_elements
                                    if e.label.scan.slices[e.slice_index].id == slice_id
                                      and e.label.owner_id == eti_id
                                   ), None)
        
        if ground_truth_expert.get(slice_id) is not None and eti_label_element is None:
            scan_id = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first().id
            eti_differences[scan_id] += 0
            eti_total[scan_id] += 1
    
combined_differences = collections.defaultdict(lambda: 0)
combined_total = collections.defaultdict(lambda: 0)

for e in best_combined_rectangular_label_elements:
    scan_id = e.label.scan.id
    slice_id = e.label.scan.slices[e.slice_index].id
    gt_rect = ground_truth_expert.get(slice_id)
    gt_rect = gt_rect if gt_rect is not None else np.array((0,0,0,0))
    combined_rect = parser.convert_to_numpy([e])[0]
    
    iou = parser.compute_intersection_over_union(combined_rect, gt_rect)
    iou = 1.0 if np.isnan(iou) else iou
    combined_differences[scan_id] += 1 if iou >= THRESHOLD else 0
    combined_total[scan_id] += 1

for slice_id in ground_truth_expert:
    for eti_id in best_combined_users_id:
        user = models.User.query.filter(models.User.id == eti_id).first()
        
        combined_label_element = next((e for e in best_combined_rectangular_label_elements
                                    if e.label.scan.slices[e.slice_index].id == slice_id
                                      and e.label.owner_id == eti_id
                                   ), None)
        
        if ground_truth_expert.get(slice_id) is not None and combined_label_element is None:
            scan_id = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first().id
            combined_differences[scan_id] += 0
            combined_total[scan_id] += 1
    
for scan_id in SCAN_IDS:
    print(' {:.3f}%'.format(eti_differences[scan_id] / eti_total[scan_id] * 100),
          '\t{:.3f}%'.format(gumed_differences[scan_id] / gumed_total[scan_id] * 100),
          '\t{:.3f}%'.format(combined_differences[scan_id] / combined_total[scan_id] * 100))

 80.392% 	75.556% 	73.750%
 78.571% 	78.125% 	77.064%
 55.446% 	41.463% 	44.966%
 53.922% 	60.976% 	52.000%
 22.878% 	57.692% 	38.568%


## Mean Average Precision @ 0.75 - między wyliczonym Ground Truth a prawdziwym Ground Truth

In [48]:
THRESHOLD = 0.75

gt_eti_differences = collections.defaultdict(lambda: 0)
gt_eti_total = collections.defaultdict(lambda: 0)

for slice_id in ground_truth_eti_best:
    scan_id = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first().id
    gt_eti = ground_truth_eti_best.get(slice_id)
    gt_eti = gt_eti if gt_eti is not None else np.array((0,0,0,0))
    gt_rect = ground_truth_expert.get(slice_id)
    gt_rect = gt_rect if gt_rect is not None else np.array((0,0,0,0))
    
    iou = parser.compute_intersection_over_union(gt_eti, gt_rect)
    iou = 1.0 if np.isnan(iou) else iou
    gt_eti_differences[scan_id] += 1 if iou >= THRESHOLD else 0
    gt_eti_total[scan_id] += 1

gt_gumed_differences = collections.defaultdict(lambda: 0)
gt_gumed_total = collections.defaultdict(lambda: 0)

for slice_id in ground_truth_gumed_best:
    scan_id = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first().id
    gt_gumed = ground_truth_gumed_best.get(slice_id)
    gt_gumed = gt_gumed if gt_gumed is not None else np.array((0,0,0,0))
    gt_rect = ground_truth_expert.get(slice_id)
    gt_rect = gt_rect if gt_rect is not None else np.array((0,0,0,0))

    iou = parser.compute_intersection_over_union(gt_gumed, gt_rect)
    iou = 1.0 if np.isnan(iou) else iou
    gt_gumed_differences[scan_id] += 1 if iou >= THRESHOLD else 0
    gt_gumed_total[scan_id] += 1

gt_combined_differences = collections.defaultdict(lambda: 0)
gt_combined_total = collections.defaultdict(lambda: 0)

for slice_id in ground_truth_combined_best:
    scan_id = models.Scan.query.join(models.Slice).filter(models.Slice.id == slice_id).first().id
    gt_combined = ground_truth_combined_best.get(slice_id)
    gt_combined = gt_combined if gt_combined is not None else np.array((0,0,0,0))
    gt_rect = ground_truth_expert.get(slice_id)
    gt_rect = gt_rect if gt_rect is not None else np.array((0,0,0,0))

    iou = parser.compute_intersection_over_union(gt_combined, gt_rect)
    iou = 1.0 if np.isnan(iou) else iou
    gt_combined_differences[scan_id] += 1 if iou >= THRESHOLD else 0
    gt_combined_total[scan_id] += 1

for scan_id in SCAN_IDS:
    print(' {:.3f}%'.format(gt_eti_differences[scan_id] / gt_eti_total[scan_id] * 100),
          '\t{:.3f}%'.format(gt_gumed_differences[scan_id] / gt_gumed_total[scan_id] * 100),
          '\t{:.3f}%'.format(gt_combined_differences[scan_id] / gt_combined_total[scan_id] * 100))

 96.429% 	100.000% 	92.857%
 95.455% 	95.455% 	90.909%
 89.286% 	92.857% 	89.286%
 85.714% 	96.429% 	89.286%
 74.000% 	78.000% 	82.000%


In [55]:
for scan in models.Scan.query.all():
    min_slice = min([slice_.position_z for slice_ in scan.slices])
    max_slice = max([slice_.position_z for slice_ in scan.slices])
    print(scan.id, len(scan.slices), min_slice, max_slice)

03647f36-d874-480b-aaec-c2c3b9d80092 28 204.893808268149 209.506774805625
b750320e-5664-4eff-b4c7-49e4f43ce6ca 28 122.112552685138 132.535646480914
7985ea16-d5a4-4bda-92db-43a3cc074216 50 185.323267364727 191.26143970512
8b5d264d-bc2f-458f-ba68-2a987d55deae 28 122.112552685138 132.535646480914
40761156-99db-4647-9b72-8e449e6cb54c 22 98.5844863206798 116.710386589234
